In [ ]:
#import ไลบรารี
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#อนุญาตการเข้าถึง Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data  = pd.read_csv('/content/drive/MyDrive/Dataset/combined_data.csv', encoding='latin1')

In [ ]:
# จัดเรียงข้อมูลแบบสวยงาม 10 แถวแรก
data.head(10)

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
5,0,larry king live at escapenumber escapenumber p...
6,0,michael pobega wrote i'm not sure if it's the ...
7,0,hi i have this error tr sample escapenumber es...
8,1,works gateway world art explore tattooing full...
9,1,upon this account he is not only very cautious...


In [ ]:
data.isnull().sum()

label    0
text     0
dtype: int64

In [ ]:
#แสดงความยาวของข้อมูล
print('length of data is',len(data))

length of data is 83448


In [ ]:
#ลดจำนวนข้อมูลให้น้อยลง
data = data.sample(n=10000)

In [ ]:
#แสดงความยาวของข้อมูล
print('length of data is',len(data))

length of data is 10000


In [ ]:
#แสดงคอลัมน์ polarity
data.label.value_counts()

1    5215
0    4785
Name: label, dtype: int64

In [ ]:
data.columns

Index(['label', 'text'], dtype='object')

In [ ]:
# ทำการลบข้อมูลซ้ำซ้อน
data = data.drop_duplicates()

In [ ]:
# คำนวณความยาวของข้อความแต่ละข้อความ
data['text_length'] = data['text'].apply(len)

In [ ]:
# ทำความสะอาดข้อความโดยลบตัวอักษรที่ไม่เป็นตัวอักษรและแปลงเป็นตัวพิมพ์เล็กทั้งหมด
import re
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

data['clean_text'] = data['text'].apply(clean_text)

In [ ]:
# ตัดคำและลบ stopwords ออกจากข้อความ
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

data['tokens'] = data['clean_text'].apply(tokenize_and_remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#แสดงขนาดความยาวของข้อมูล
print('length of data is', len(data))

length of data is 10000


In [ ]:
# จัดเรียงข้อมูลแบบสวยงาม 10 แถวแรก
data.head(10)

,label,text,text_length,clean_text,tokens
11077,0,start date : 1 / 6 / 02 ; hourahead hour : 10 ...,496,start date hourahead hour no ancilla...,"[start, date, hourahead, hour, ancillary, sche..."
16974,0,alert name bush new war funding bill unaccepta...,873,alert name bush new war funding bill unaccepta...,"[alert, name, bush, new, war, funding, bill, u..."
82594,1,after all the genuine stuff escapenumber with ...,903,after all the genuine stuff escapenumber with ...,"[genuine, stuff, escapenumber, ripoffs, p, e, ..."
56791,0,alert name bush white house visitor logs exemp...,930,alert name bush white house visitor logs exemp...,"[alert, name, bush, white, house, visitor, log..."
41539,1,dear valued member if you are buying drugs on ...,716,dear valued member if you are buying drugs on ...,"[dear, valued, member, buying, drugs, web, fre..."
35030,1,Â ÂÂ© ÂÂ© Â Â Âe ÂeÂ ÂÂ¦ ÂÃ¶ ÂÃ¶ Â...,271,e e zmb e mailfhttp gjsg com neko ...,"[e, e, zmb, e, mailfhttp, gjsg, com, neko, mil..."
56606,1,total number of overweight adults escapenumber...,2700,total number of overweight adults escapenumber...,"[total, number, overweight, adults, escapenumb..."
76976,1,it is going to break out camp aign for chvccur...,245,it is going to break out camp aign for chvccur...,"[going, break, camp, aign, chvccurrent, escape..."
6977,1,the internet sector is hot again sea rch for c...,253,the internet sector is hot again sea rch for c...,"[internet, sector, hot, sea, rch, chvccurrent,..."
16936,1,the best all new adult dating site on the net ...,2480,the best all new adult dating site on the net ...,"[best, new, adult, dating, site, net, check, y..."


In [ ]:
data.to_csv('/content/drive/My Drive/Dataset/dataset.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# แบ่งข้อมูลเป็น X และ y
X = data['tokens'].apply(lambda x: ' '.join(x))
y = data['label']

In [ ]:
# แบ่งชุดข้อมูลเป็น training set และ testing set โดยให้ testing set มีขนาด 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# สร้างเวกเตอร์คุณลักษณะโดยใช้ CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
# สร้างและประเมินโมเดล SVM
svm_model = SVC()
svm_scores = cross_val_score(svm_model, X_train_vectorized, y_train, cv=5)

In [ ]:
# สร้างและประเมินโมเดล ANN
ann_model = MLPClassifier()
ann_scores = cross_val_score(ann_model, X_train_vectorized, y_train, cv=5)

In [ ]:
# สร้างและประเมินโมเดล Naive Bayes
nb_model = MultinomialNB()
nb_scores = cross_val_score(nb_model, X_train_vectorized, y_train, cv=5)

In [ ]:
# สร้างและประเมินโมเดล Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_scores = cross_val_score(lr_model, X_train_vectorized, y_train, cv=5)

In [ ]:
# สร้างและประเมินโมเดล K-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier()
knn_scores = cross_val_score(knn_model, X_train_vectorized, y_train, cv=5)

In [ ]:
# แสดงผลคะแนน cross validation score ของแต่ละโมเดล
print("SVM Cross Validation Scores:", svm_scores)
print("ANN Cross Validation Scores:", ann_scores)
print("Naive Bayes Cross Validation Scores:", nb_scores)
print("Logistic Regression Cross Validation Scores:", lr_scores)
print("K-Nearest Neighbors (KNN) Cross Validation Scores:", knn_scores)

SVM Cross Validation Scores: [0.853125 0.820625 0.854375 0.843125 0.84    ]
ANN Cross Validation Scores: [0.97125  0.968125 0.97125  0.97     0.97625 ]
Naive Bayes Cross Validation Scores: [0.97     0.970625 0.961875 0.970625 0.965625]
Logistic Regression Cross Validation Scores: [0.975625 0.963125 0.969375 0.963125 0.971875]
K-Nearest Neighbors (KNN) Cross Validation Scores: [0.87125  0.855    0.86375  0.855625 0.8825  ]


# 3. การประเมินโมเดล (Model Evaluation)

####SVM

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# ประเมินโมเดล SVM
svm_model.fit(X_train_vectorized, y_train)
svm_predictions = svm_model.predict(X_test_vectorized)

In [ ]:
# แสดงค่า Classification Report
print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.70      0.82       965
           1       0.78      0.99      0.87      1035

    accuracy                           0.85      2000
   macro avg       0.88      0.84      0.84      2000
weighted avg       0.88      0.85      0.84      2000



In [ ]:
# แสดงค่า Confusion Matrix
svm_conf_matrix = confusion_matrix(y_test, svm_predictions)
print("SVM Confusion Matrix:")
print(svm_conf_matrix)

SVM Confusion Matrix:
[[ 673  292]
 [  13 1022]]


####ANN

In [ ]:
# ประเมินโมเดล ANN
ann_model.fit(X_train_vectorized, y_train)
ann_predictions = ann_model.predict(X_test_vectorized)

In [ ]:
# แสดงค่า Classification Report
print("ANN Classification Report:")
print(classification_report(y_test, ann_predictions))

ANN Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       965
           1       0.97      0.99      0.98      1035

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [ ]:
# แสดงค่า Confusion Matrix
ann_conf_matrix = confusion_matrix(y_test, ann_predictions)
print("ANN Confusion Matrix:")
print(ann_conf_matrix)

ANN Confusion Matrix:
[[ 937   28]
 [  15 1020]]


####Naive Bayes

In [ ]:
# ประเมินโมเดล Naive Bayes
nb_model.fit(X_train_vectorized, y_train)
nb_predictions = nb_model.predict(X_test_vectorized)

In [ ]:
# แสดงค่า Classification Report
print("Classification Report for Naive Bayes:")
print(classification_report(y_test, nb_predictions))

Classification Report for Naive Bayes:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       965
           1       0.99      0.96      0.97      1035

    accuracy                           0.97      2000
   macro avg       0.97      0.97      0.97      2000
weighted avg       0.97      0.97      0.97      2000



In [ ]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for Naive Bayes:")
print(confusion_matrix(y_test, nb_predictions))

Confusion Matrix for Naive Bayes:
[[952  13]
 [ 43 992]]


####Logistic Regression

In [ ]:
# ประเมินโมเดล Logistic Regression
lr_model.fit(X_train_vectorized, y_train)
lr_predictions = lr_model.predict(X_test_vectorized)

In [ ]:
# แสดงค่า Classification Report
print("Classification Report for Logistic Regression:")
print(classification_report(y_test, lr_predictions))

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       965
           1       0.97      0.99      0.98      1035

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [ ]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for Logistic Regression:")
print(confusion_matrix(y_test, lr_predictions))

Confusion Matrix for Logistic Regression:
[[ 932   33]
 [  10 1025]]


####K-Nearest Neighbors (KNN)

In [ ]:
# ประเมินโมเดล K-Nearest Neighbors (KNN)
knn_model.fit(X_train_vectorized, y_train)
knn_predictions = knn_model.predict(X_test_vectorized)

In [ ]:
# แสดงค่า Classification Report
print("Classification Report for K-Nearest Neighbors (KNN):")
print(classification_report(y_test, knn_predictions))

Classification Report for K-Nearest Neighbors (KNN):
              precision    recall  f1-score   support

           0       0.95      0.77      0.85       965
           1       0.82      0.96      0.88      1035

    accuracy                           0.87      2000
   macro avg       0.88      0.86      0.86      2000
weighted avg       0.88      0.87      0.86      2000



In [ ]:
# แสดงค่า Confusion Matrix
print("Confusion Matrix for K-Nearest Neighbors (KNN):")
print(confusion_matrix(y_test, knn_predictions))

Confusion Matrix for K-Nearest Neighbors (KNN):
[[741 224]
 [ 43 992]]


#4. การนำโมเดลไปใช้งาน (Deploy Model)

####บันทึกโมเดลที่ดีกว่า

In [ ]:
import pickle

# บันทึกโมเดล ANN ที่ดีที่สุดด้วยไลบรารี pickle
with open('/content/drive/MyDrive/Dataset/best_ann_model.pkl', 'wb') as file:
    pickle.dump(svm_model, file)

####ทดสอบโหลดโมเดลไปใช้งาน โดยการอ่านข้อมูลเข้ามา 1 ตัวอย่าง สกัดคุณลักษณะและทำนายผลลัพธ์

In [ ]:
# ทดสอบโหลดโมเดล ANN ที่บันทึกไว้
loaded_ann_model = None
with open('/content/drive/MyDrive/Dataset/best_ann_model.pkl', 'rb') as file:
    loaded_ann_model = pickle.load(file)

In [ ]:
# ข้อความตัวอย่าง
example_textham = [" it works fine with escapenumber escapenumber escapenumber on my fedora core escapenumber perl e 'for abc\\n def hij\\n print warn tell stdout \\n ' escapenumber abc escapenumber escapenumber defhij escapenumber does not seem to be accurate on this platform uname a cygwin nt escapenumber escapenumber escapelong escapenumber escapenumber escapenumber escapenumber escapenumber escapenumber escapenumber escapenumber escapenumber escapenumber escapenumber escapenumber iescapenumber cygwin perl e 'for abc\\n def hij\\n print warn tell stdout \\n ' escapenumber abc escapenumber escapenumber defhij escapenumber do you yahoo tired of spam yahoo mail has the best spam protection around http mail yahoo com to unsubscribe e mail beginners unsubscribe perl org for additional commands e mail beginners help perl org http learn perl org "]
example_textSpam = [" your your credit report doesn't matter to us if you own real estate and want immediate ready money to spend any way you like or simply need to lower your entire payment by a third or more here is the deal we can offer you tonight hurry this offer will expire this evening escapenumber escapenumber loan and even more after further review our lenders have established the lowest monthly payments hurry when best deal is gone it is gone simply complete this short form don't worry about approval your your credit report will not disqualify you http luvuiiksheap com "]

# ใช้ CountVectorizer ในการแปลงข้อความตัวอย่างเป็นเวกเตอร์คุณลักษณะ
example_text_vectorized = vectorizer.transform(example_textSpam)

# ทำนายผลลัพธ์โดยใช้โมเดล ANN ที่โหลดมา
prediction = loaded_ann_model.predict(example_text_vectorized)

# แสดงผลลัพธ์ที่ทำนาย
if prediction == 1:
    print("Predicted Label (Using Loaded ANN Model): Spam")
else:
    print("Predicted Label (Using Loaded ANN Model): Ham")


Predicted Label (Using Loaded ANN Model): Spam
